In [26]:
import tensorflow as ts # TODO

ImportError: No module named tensorflow

In [16]:
import numpy
import matplotlib.pyplot as plt

ImportError: No module named matplotlib.pyplot

In [17]:
# Parameters
learning_rate = 0.005
training_epochs = 1000
batch_size = 50

In [32]:
def mackey_glass(t, t25):
    return 0.9 * t + 0.2*t25/(1+t25**10)
fdata = [1.5]
for i in range(1525):
    fdata.append(mackey_glass(fdata[i], fdata[i - 25] if i > 25 else 0))

In [41]:
def create_mg_series(i, data):
    return numpy.asarray([data[i-20], data[i-15], data[i-10], data[i-5], data[i]])
train_X = numpy.asarray(map(lambda x: create_mg_series(x, fdata), range(301, 1300)))
train_Y = numpy.asarray(map(lambda x: fdata[x+5], range(301, 1300)))
test_X = numpy.asarray(map(lambda x: create_mg_series(x, fdata), range(1301, 1500)))
test_Y = numpy.asarray(map(lambda x: fdata[x+5], range(1301, 1500)))
n_samples = train_X.shape[0]

In [20]:
# X Y
x = tf.placeholder(tf.float32, [None, 5], name='x')  # 5 inputs
y = tf.placeholder(tf.float32, name='y')  # 1 output

# hidden layer 1
W1 = tf.Variable(tf.truncated_normal([5, 8], stddev=0.03), name='W1')
b1 = tf.Variable(tf.truncated_normal([8]), name='b1')

# hidden layer 2
W2 = tf.Variable(tf.truncated_normal([8, 5], stddev=0.03), name='W2')
b2 = tf.Variable(tf.truncated_normal([5]), name='b2')

NameError: name 'tf' is not defined

In [ ]:
# output hidden layer 1
hidden_out = tf.nn.relu(tf.add(tf.matmul(x, W1), b1))

# total output
y_ = tf.nn.relu(tf.add(tf.matmul(hidden_out, W2), b2))

In [ ]:
mse = tf.losses.mean_squared_error(y, y_)

In [ ]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(mse)

In [ ]:
# initialize variables
init_op = tf.global_variables_initializer()

In [ ]:
# accuracy for the test set
accuracy = tf.reduce_mean(tf.square(tf.subtract(y, y_))) # or could use tf.losses.mean_squared_error

In [25]:
with tf.Session() as sess:
    sess.run(init_op)
    batches = int(len(train_Y) / batch_size) 
    # Fit all training data
    for epoch in range(training_epochs):
        for (x, y) in zip(train_X, train_Y):
            sess.run(optimizer, feed_dict={X: x, Y: y})

        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            c = sess.run(mse, feed_dict={X: train_X, Y:train_Y})
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(c), \
                "W1=", sess.run(W1), "b1=", sess.run(b1),
                "W2=", sess.run(W2), "b2=", sess.run(b2))
    training_cost = sess.run(mse, feed_dict={X: train_X, Y: train_Y})
    print("Training cost=", training_cost, "W=", sess.run(W), "b=", sess.run(b), '\n')

    # Graphic display
    plt.plot(train_X, train_Y, 'ro', label='Original data')
    plt.legend()
    plt.show()

    print("Testing... (Mean square loss Comparison)")
    testing_cost = sess.run(
        tf.reduce_sum(tf.pow(y_ - Y, 2)) / (2 * test_X.shape[0]),
        feed_dict={X: test_X, Y: test_Y})  # same function as cost above
    print("Testing cost=", testing_cost)
    print("Absolute mean square loss difference:", abs(
        training_cost - testing_cost))

    plt.plot(test_X, test_Y, 'bo', label='Testing data')
    plt.legend()
    plt.show()

NameError: name 'tf' is not defined